In [1]:
import socket
import datetime
from dateutil import parser

In [2]:
def getSynchronizedTime(clock_server_connection):
    #Time at which client sent a request to clockServer
    clock_server_connection.send("SYNC_TIME".encode('utf-8'))
    clock_server_request_time = datetime.datetime.now().timestamp()   #T0
    synced_time_str = clock_server_connection.recv(2048).decode()
    clock_data_recv = parser.parse(synced_time_str)
    #Time at which client received a response from the clockServer
    clock_server_response_time = datetime.datetime.now().timestamp() #T1
    latency = clock_server_response_time - clock_server_request_time
    client_time = clock_data_recv + datetime.timedelta(seconds= latency/2)
    return client_time

In [3]:
host = '127.0.0.1'
clock_port = 8080
certificate_port = 5000

clock_server_connection = socket.socket()
clock_server_connection.connect((host, clock_port))

#connects to certificate granting server
synced_time = getSynchronizedTime(clock_server_connection)
print('Synced time', synced_time)
clock_server_connection.close()

Synced time 2023-04-24 21:25:02.186973
